In [ ]:
import os
import warnings

from ticoi import example
from ticoi.core import process, process_blocks_refine, save_cube_parameters
from ticoi.cube_data_classxr import CubeDataClass
from ticoi.cube_writer import CubeResultsWriter

warnings.filterwarnings("ignore")

## 1) Parameters

Path of the input and output data (to change)

In [ ]:
cube_name = example.get_path("ITS_LIVE_Lowell_Lower", overwrite=True)
path_save = os.path.join(
    os.path.abspath(os.path.join(os.path.dirname(os.path.abspath(__file__)), "..", "..", "..")),
    "examples",
    "results",
    "cube",
)  # path where to save our results
result_fn = "Lowell_example"  # Name of the netCDF file to be created

Main parameters (to change)

In [ ]:
## ----------------------- saving options --------------------- ##
#Output to save or not
save = False  # If True, save TICOI results to a netCDF file
save_mean_velocity = False  # Save a .tiff file with the mean resulting velocities, as an example

# What results must be returned from TICOI processing (can be a list of both)
#   - 'invert' for the results of the inversion, corresponding to Cumulative displacement time series
#   - 'interp' for the results of the interpolation
returned = ["invert", "interp"]

## ----------------------- TICOI main parameters --------------------- ##
# We advice the user to change only the following parameter, the other parameters stored in a dictionary can be kept as it is for a first use
regu = "1accelnotnull"  # Regularization method.s to be used (for each flag if flag is not None) : 1 minimize the acceleration, '1accelnotnull' minize the distance with an apriori on the acceleration computed over a spatio-temporal filtering of the cube
coef = 100  # Regularization coefficient.s to be used (for each flag if flag is not None)
delete_outlier = None  # Delete data, check possible parameters in README_possible_parameters
apriori_weight = True

proj = "EPSG:3413"  # EPSG system of the given coordinates
nb_cpu = 12  # Number of CPU to be used for parallelization
block_size = 0.5  # Maximum sub-block size (in GB) for the 'block_process' TICOI processing method

Dictionary to store the different parameters

In [ ]:
load_kwargs = {
    "chunks": {},
    "proj": proj,  # EPSG system of the given coordinates
}

preData_kwargs = {
    "delete_outliers": delete_outlier,  # Delete data with a poor quality indicator (if int), or with aberrant direction ('vvc_angle')
    "regu": regu,  # Regularization method.s to be used (for each flag if flag is not None) : 1 minimize the acceleration, '1accelnotnull' minize the distance with an apriori on the acceleration computed over a spatio-temporal filtering of the cube
    "proj": proj,  # EPSG system of the given coordinates
}
## ---------------- Inversion and interpolation parameters ----------------- ##
inversion_kwargs = {
    "coef": coef,  # Regularization coefficient.s to be used (for each flag if flag is not None)
    "apriori_weight": apriori_weight,  # If True, use apriori weights
    "detect_temporal_decorrelation": True,  # If True, the first inversion will use only velocity observations with small temporal baselines, to detect temporal decorelation
    "result_quality": "X_contribution",  # Criterium used to evaluate the quality of the results ('Norm_residual', 'X_contribution')
    "path_save": path_save,  # Path where to store the results
    "proj": proj,
}

## 2) Cube processing

In [ ]:
# Load the first cube
cube = CubeDataClass()
cube.load(cube_name, **load_kwargs)

#Prepare interpolation dates
first_date_interpol, last_date_interpol = cube.prepare_interpolation_date()
inversion_kwargs.update({"first_date_interpol": first_date_interpol, "last_date_interpol": last_date_interpol})

# TICOI processing
# The data cube is subdivided in smaller cubes computed one after the other in a synchronous manner (uses async)
# TICOI computation is then parallelized among those cubes
result = process_blocks_refine(
    cube,
    nb_cpu=nb_cpu,
    block_size=block_size,
    preData_kwargs=preData_kwargs,
    inversion_kwargs=inversion_kwargs,
    returned=returned,
)

## 3) Cube saving

In [ ]:
if save:
    if "invert" in returned:
        source, sensor = save_cube_parameters(cube, load_kwargs, preData_kwargs, inversion_kwargs, returned="invert")
    if "interp" in returned:
        source_interp, sensor = save_cube_parameters(
            cube, load_kwargs, preData_kwargs, inversion_kwargs, returned="interp"
        )

    several = isinstance(returned, list) and len(returned) >= 2
    writer = CubeResultsWriter(cube)

    if "invert" in returned:
        cube_invert = writer.write_result_tico(
            result["invert"] if several else result,
            source,
            sensor,
            result_quality=inversion_kwargs["result_quality"],
            filename=f"{result_fn}_invert1" if several else result_fn,
            savepath=path_save if save else None,
            verbose=inversion_kwargs["verbose"],
        )
    if "interp" in returned:
        cube_interp = writer.write_result_ticoi(
            result["interp"] if several else result,
            source_interp,
            sensor,
            result_quality=inversion_kwargs["result_quality"],
            filename=f"{result_fn}_interp1" if several else result_fn,
            savepath=path_save if save else None,
            verbose=inversion_kwargs["verbose"],
        )

## 4) Temporal average of the cube (optional)

In [ ]:
# Plot the mean velocity as an example
if save_mean_velocity and cube_interp is not None:
    cube_interp.average_cube(return_format="geotiff", return_variable=["vv"], save=True, path_save=path_save)